In [2]:
from smolagents import HfApiModel, CodeAgent
from dotenv import load_dotenv
import os

d:\Genai.labs assignment\assignment\Ecommerce_Assistant_Challenge 2025\genailabs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

import os
import io
import IPython.display
from PIL import Image
import base64

from dotenv import load_dotenv, find_dotenv

# Specify the path to your .env file
dotenv_path = r"D:\Genai.labs assignment\assignment\Ecommerce_Assistant_Challenge 2025\.env"

# Load environment variables from the specified.env file
_ = load_dotenv() # read local .env file

from huggingface_hub import login

login(os.getenv("HUGGINGFACE_API_KEY"))

In [10]:
from smolagents import tool
from smolagents import CodeAgent, UserInputTool, DuckDuckGoSearchTool, HfApiModel
import requests
from typing import List, Dict, Any, Optional, Union
import pandas as pd
import logging

In [3]:
# Define base URLs for both services
PRODUCT_SERVICE_URL = os.getenv("PRODUCT_SERVICE_URL", "http://localhost:8001")
ORDER_SERVICE_URL = os.getenv("ORDER_SERVICE_URL", "http://localhost:8002")

In [4]:
PRODUCT_SERVICE_URL

'http://localhost:8001'

In [7]:
search_products("Shoes",3)

NameError: name 'search_products' is not defined

# Tools

In [14]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ==========================================
# Product Service Tools
# ==========================================

@tool
def search_products(query: str, top_k: int = 5) -> List[Dict[str, Any]]:
    """
    Search for products based on a text query using RAG.
    
    Args:
        query: Search query string
        top_k: Number of results to return (default: 5)
        
    Returns:
        List of product dictionaries containing details like title, description, price, etc.
    """
    try:
        response = requests.get(
            f"{PRODUCT_SERVICE_URL}/search",
            params={"query": query, "top_k": top_k}
        )
        response.raise_for_status()
        data = response.json()
        
        return data.get("results", [])
    except Exception as e:
        logger.error(f"Error searching products: {str(e)}")
        return []

@tool
def search_product_by_category(category: str, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
    """
    Search for products in a specific category.
    
    Args:
        category: Category to search in
        query: Search query string
        top_k: Number of results to return (default: 5)
        
    Returns:
        List of product dictionaries containing details like title, description, price, etc.
    """
    try:
        response = requests.get(
            f"{PRODUCT_SERVICE_URL}/search/category",
            params={"category": category, "query": query, "top_k": top_k}
        )
        response.raise_for_status()
        data = response.json()
        
        return data.get("results", [])
    except Exception as e:
        logger.error(f"Error searching products by category: {str(e)}")
        return []

@tool
def get_top_rated_products(category: Optional[str] = None, min_rating: float = 4.5, top_k: int = 5) -> List[Dict[str, Any]]:
    """
    Get top-rated products, optionally filtered by category.
    
    Args:
        category: Category to filter by (optional)
        min_rating: Minimum rating threshold (default: 4.5)
        top_k: Number of results to return (default: 5)
        
    Returns:
        List of top-rated product dictionaries
    """
    try:
        params = {"min_rating": min_rating, "top_k": top_k}
        if category:
            params["category"] = category
            
        response = requests.get(
            f"{PRODUCT_SERVICE_URL}/top-rated",
            params=params
        )
        response.raise_for_status()
        data = response.json()
        
        return data.get("results", [])
    except Exception as e:
        logger.error(f"Error fetching top-rated products: {str(e)}")
        return []

@tool
def get_product_details(product_id: int) -> Dict[str, Any]:
    """
    Get detailed information about a specific product.
    
    Args:
        product_id: ID of the product
        
    Returns:
        Dictionary containing product details
    """
    try:
        response = requests.get(f"{PRODUCT_SERVICE_URL}/product/{product_id}")
        response.raise_for_status()
        return response.json()
    except Exception as e:
        logger.error(f"Error fetching product details: {str(e)}")
        return {}

@tool
def get_specific_instrument_details(instrument_type: str) -> List[Dict[str, Any]]:
    """
    Get information about a specific type of musical instrument.
    
    Args:
        instrument_type: Type of instrument (e.g., 'guitar', 'piano', 'drums')
        
    Returns:
        List of dictionaries containing instrument details
    """
    # This is a specialized search focused on instruments
    try:
        return search_products(instrument_type, top_k=3)
    except Exception as e:
        logger.error(f"Error fetching specific instrument details: {str(e)}")
        return []

@tool
def compare_products(product_ids: List[int]) -> List[Dict[str, Any]]:
    """
    Compare multiple products side by side.
    
    Args:
        product_ids: List of product IDs to compare
        
    Returns:
        List of dictionaries containing product details for comparison
    """
    products = []
    for product_id in product_ids:
        try:
            product = get_product_details(product_id)
            if product:
                products.append(product)
        except Exception as e:
            logger.error(f"Error fetching product {product_id} for comparison: {str(e)}")
    
    return products

# ==========================================
# Order Service Tools
# ==========================================

@tool
def get_customer_orders(customer_id: int) -> List[Dict[str, Any]]:
    """
    Get all orders for a specific customer.
    
    Args:
        customer_id: Customer ID
        
    Returns:
        List of dictionaries containing order information
    """
    try:
        response = requests.get(f"{ORDER_SERVICE_URL}/customer/{customer_id}")
        response.raise_for_status()
        data = response.json()
        
        return data.get("orders", [])
    except Exception as e:
        logger.error(f"Error fetching customer orders: {str(e)}")
        return []

@tool
def get_customer_recent_order(customer_id: int) -> Dict[str, Any]:
    """
    Get the most recent order for a specific customer.
    
    Args:
        customer_id: Customer ID
        
    Returns:
        Dictionary containing the most recent order details
    """
    try:
        response = requests.get(f"{ORDER_SERVICE_URL}/customer/{customer_id}/recent")
        response.raise_for_status()
        data = response.json()
        
        return data.get("order", {})
    except Exception as e:
        logger.error(f"Error fetching recent order: {str(e)}")
        return {}

@tool
def get_customer_product_orders(customer_id: int, product_keyword: str) -> List[Dict[str, Any]]:
    """
    Get orders containing a specific product for a customer.
    
    Args:
        customer_id: Customer ID
        product_keyword: Keyword to search in product name or category
        
    Returns:
        List of dictionaries containing matching order information
    """
    try:
        response = requests.get(
            f"{ORDER_SERVICE_URL}/customer/{customer_id}/product",
            params={"product_keyword": product_keyword}
        )
        response.raise_for_status()
        data = response.json()
        
        return data.get("orders", [])
    except Exception as e:
        logger.error(f"Error fetching product orders: {str(e)}")
        return []

@tool
def get_high_priority_orders(limit: int = 5) -> List[Dict[str, Any]]:
    """
    Get recent high-priority orders.
    
    Args:
        limit: Maximum number of orders to return (default: 5)
        
    Returns:
        List of dictionaries containing high-priority order information
    """
    try:
        response = requests.get(
            f"{ORDER_SERVICE_URL}/high-priority",
            params={"limit": limit}
        )
        response.raise_for_status()
        data = response.json()
        
        return data.get("orders", [])
    except Exception as e:
        logger.error(f"Error fetching high-priority orders: {str(e)}")
        return []

@tool
def get_sales_by_category() -> List[Dict[str, Any]]:
    """
    Get total sales data aggregated by product category.
    
    Returns:
        List of dictionaries with category and sales data
    """
    try:
        response = requests.get(f"{ORDER_SERVICE_URL}/total-sales-by-category")
        response.raise_for_status()
        data = response.json()
        
        return data.get("categories", [])
    except Exception as e:
        logger.error(f"Error fetching sales by category: {str(e)}")
        return []

@tool
def get_high_profit_products(min_profit: float = 100.0) -> List[Dict[str, Any]]:
    """
    Get high-profit products.
    
    Args:
        min_profit: Minimum profit threshold (default: 100.0)
        
    Returns:
        List of dictionaries containing high-profit product order information
    """
    try:
        response = requests.get(
            f"{ORDER_SERVICE_URL}/high-profit-products",
            params={"min_profit": min_profit}
        )
        response.raise_for_status()
        data = response.json()
        
        return data.get("products", [])
    except Exception as e:
        logger.error(f"Error fetching high-profit products: {str(e)}")
        return []

@tool
def get_shipping_cost_summary() -> Dict[str, float]:
    """
    Get shipping cost summary (average, min, max).
    
    Returns:
        Dictionary with shipping cost statistics
    """
    try:
        response = requests.get(f"{ORDER_SERVICE_URL}/shipping-cost-summary")
        response.raise_for_status()
        return response.json()
    except Exception as e:
        logger.error(f"Error fetching shipping cost summary: {str(e)}")
        return {}

@tool
def get_profit_by_gender() -> List[Dict[str, Any]]:
    """
    Get total profit aggregated by customer gender.
    
    Returns:
        List of dictionaries with gender and profit data
    """
    try:
        response = requests.get(f"{ORDER_SERVICE_URL}/profit-by-gender")
        response.raise_for_status()
        data = response.json()
        
        return data.get("genders", [])
    except Exception as e:
        logger.error(f"Error fetching profit by gender: {str(e)}")
        return []

# ==========================================
# Combined / Helper Tools
# ==========================================

@tool
def check_product_availability(product_name: str, customer_id: Optional[int] = None) -> Dict[str, Any]:
    """
    Check if a product is available and if the customer has ordered it before.
    
    Args:
        product_name: Name of the product to check
        customer_id: Optional customer ID to check order history
        
    Returns:
        Dictionary with availability information and order history if applicable
    """
    result = {
        "product_found": False,
        "product_details": None,
        "previously_ordered": False,
        "previous_orders": []
    }
    
    # Search for the product
    products = search_products(product_name)
    if products:
        result["product_found"] = True
        result["product_details"] = products[0]  # Get the first match
    
    # Check if customer has ordered this product before
    if customer_id and result["product_found"]:
        product_orders = get_customer_product_orders(customer_id, product_name)
        if product_orders:
            result["previously_ordered"] = True
            result["previous_orders"] = product_orders
    
    return result

@tool
def recommend_similar_products(product_name: str, top_k: int = 3) -> List[Dict[str, Any]]:
    """
    Recommend products similar to the specified product.
    
    Args:
        product_name: Name of the reference product
        top_k: Number of recommendations to return (default: 3)
        
    Returns:
        List of dictionaries containing recommended product details
    """
    # First find the product to establish category and features
    products = search_products(product_name, top_k=1)
    
    if not products:
        return []
    
    product = products[0]
    category = product.get("main_category", "")
    
    # If we have a category, search within that category
    if category:
        return search_product_by_category(category, product_name, top_k=top_k)
    else:
        # Otherwise just do a regular search
        return search_products(product_name, top_k=top_k+1)[1:]  # Skip the first result which is the product itself

@tool
def get_customer_order_summary(customer_id: int) -> Dict[str, Any]:
    """
    Get a summary of a customer's order history.
    
    Args:
        customer_id: Customer ID
        
    Returns:
        Dictionary with order summary statistics
    """
    orders = get_customer_orders(customer_id)
    
    if not orders:
        return {
            "customer_id": customer_id,
            "total_orders": 0,
            "message": "No order history found for this customer."
        }
    
    # Convert to DataFrame for easier analysis
    df = pd.DataFrame(orders)
    
    # Calculate summary statistics
    summary = {
        "customer_id": customer_id,
        "total_orders": len(df),
        "total_spend": round(df["Sales"].sum(), 2) if "Sales" in df.columns else 0,
        "average_order_value": round(df["Sales"].mean(), 2) if "Sales" in df.columns else 0,
        "total_shipping_cost": round(df["Shipping_Cost"].sum(), 2) if "Shipping_Cost" in df.columns else 0,
    }
    
    # Add most frequent product categories if available
    if "Product_Category" in df.columns:
        category_counts = df["Product_Category"].value_counts().to_dict()
        summary["top_categories"] = [
            {"category": category, "count": count} 
            for category, count in list(category_counts.items())[:3]
        ]
    
    # Add most recent order date if available
    if "Order_Date" in df.columns:
        df["Order_Date"] = pd.to_datetime(df["Order_Date"])
        summary["most_recent_order_date"] = df["Order_Date"].max().strftime("%Y-%m-%d")
        summary["first_order_date"] = df["Order_Date"].min().strftime("%Y-%m-%d")
    
    return summary

@tool
def search_and_get_rating_info(query: str) -> Dict[str, Any]:
    """
    Get rating information for products matching the search query.
    
    Args:
        query: Search query string
        
    Returns:
        Dictionary with rating statistics for matching products
    """
    products = search_products(query)
    
    if not products:
        return {
            "query": query,
            "products_found": 0,
            "message": "No products found matching the query."
        }
    
    # Extract ratings and count them
    ratings = [p.get("average_rating", 0) for p in products if p.get("average_rating") is not None]
    
    return {
        "query": query,
        "products_found": len(products),
        "average_rating": round(sum(ratings) / len(ratings), 1) if ratings else 0,
        "highest_rated_product": max(products, key=lambda p: p.get("average_rating", 0)) if products else None,
        "lowest_rated_product": min(products, key=lambda p: p.get("average_rating", 0) if p.get("average_rating") is not None else float('inf')) if products else None
    }

# Agentic Setup

In [6]:
model = HfApiModel(
    "Qwen/Qwen2.5-7B-Instruct",
    provider="together", # Choose a specific inference provider
    max_tokens=4096,
    temperature=0.1
)

In [7]:
task = """Your task is to be a good assistant and help the user with their queries regarding the product and order services.
if the user dont provide the requried information, ask them to provide the required information. For example, if the user ask for the order history, ask them to provide the customer id.
Dont assume anything, just ask the user to provide the required information.
"""

In [8]:
agent = CodeAgent(
    model=model,
    tools=[search_products,
      search_product_by_category,
      get_top_rated_products, 
      get_product_details,
      get_specific_instrument_details, 
      compare_products, 
      get_customer_orders,
      get_customer_recent_order, 
      get_customer_product_orders,
      get_high_priority_orders,
      get_sales_by_category,
      get_high_profit_products,
      get_shipping_cost_summary,
      get_profit_by_gender,
      check_product_availability,
      recommend_similar_products,
      get_customer_order_summary,
      search_and_get_rating_info,
      UserInputTool(),
     ],
    max_steps=5,
    additional_authorized_imports=["pandas", "numpy"],
    verbosity_level=2
)

NameError: name 'search_products' is not defined

In [9]:
agent.run(
    task=task,
    additional_args={"user_query": """What are the details of my most recent order?"""}
)

NameError: name 'agent' is not defined

In [ ]:
from smolagents import ToolCallingAgent

model = HfApiModel(
    "Qwen/Qwen2.5-7B-Instruct",
    provider="together", # Choose a specific inference provider
    max_tokens=4096,
    temperature=0.6
)
tool_agent = ToolCallingAgent(
    model=model,
    tools=[search_products,
      search_product_by_category,
      get_top_rated_products, 
      get_product_details,
      get_specific_instrument_details, 
      compare_products, 
      get_customer_orders,
      get_customer_recent_order, 
      get_customer_product_orders,
      get_high_priority_orders,
      get_sales_by_category,
      get_high_profit_products,
      get_shipping_cost_summary,
      get_profit_by_gender,
      check_product_availability,
      recommend_similar_products,
      get_customer_order_summary,
      search_and_get_rating_info,
      UserInputTool()
     ],
    max_steps=10,
)

tool_agent.run(
    task=task,
    additional_args={"user_query": """What are the details of my most recent order?"""}
)

In [26]:
tool_agent.run(
    task=task,
    additional_args={"user_query": """What are the details of my most recent order?"""}
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Your task is to be a good assistant and help the user with their queries regarding the product and order        │
│ services.                                                                                                       │
│ if the user dont provide the requried information, ask them to provide the required information. For example,   │
│ if the user ask for the order history, ask them to provide the customer id.                                     │
│ Dont assume anything, just ask the user to provide the required information.                                    │
│                                                                                                                 │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'user_query': 'What are the details of my most recent order?'}.                                                │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-7B-Instruct ─────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'get_customer_recent_order' with arguments: {'customer_id': 1}                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ERROR:__main__:Error fetching recent order: 404 Client Error: Not Found for url: http://localhost:8002/customer/1/recent


Observations: {}

...

Error while parsing tool call from model output: The model output does not contain any JSON blob.

[Step 10: Duration 0.82 seconds| Input tokens: 45,271 | Output tokens: 245]

Reached max steps.

[Step 11: Duration 1.26 seconds| Input tokens: 46,388 | Output tokens: 268]

'To fetch the details of your most recent order, I need your customer ID. Could you please provide it?'

In [ ]:
from smolagents import CodeAgent, UserInputTool, DuckDuckGoSearchTool, HfApiModel

agent = CodeAgent(
    model=HfApiModel(),
    tools=[
      DuckDuckGoSearchTool(),
      UserInputTool(),      # ← ask the human when needed
    ],
    add_base_tools=True,
)

# If the LLM decides you need to ask the user something,
# it will generate a call like:
#    user_input(question="What query do you want to search?")
# and that will block on input(...) in the console.
agent.run("Search for ‘latest HF releases’, but ask me which model to use first.")


d:\Genai.labs assignment\assignment\Ecommerce_Assistant_Challenge 2025\genailabs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for ‘latest HF releases’, but ask me which model to use first.                                           │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  model_choice = user_input("Which model are you interested in? (e. g., GPT, BERT, etc.)")                         
  print(f"User selected model: {model_choice}")                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Closing server running on port: 7860
